In [1]:
import pandas as pd
import numpy as np
#import win32com.client as win32
import pytesseract
import matplotlib.pyplot as plt
import nltk
import cv2
import os
import json
import torch
import pickle
from tqdm import tqdm
#from unidecode import unidecode

In [2]:
os.chdir(r'C:\Users\puboggavarapu\Desktop\skills&studies\kaggle\covid_19')

### Getting full text for all files that have full text available

In [4]:
metadata = pd.read_csv('metadata.csv')

In [5]:
metadata['full_text_file'].value_counts()

custom_license        20873
comm_use_subset        8803
noncomm_use_subset     2133
biorxiv_medrxiv        1020
Name: full_text_file, dtype: int64

### Example

In [7]:
#Example
path = './' + metadata.loc[3,'full_text_file'] + '/' + metadata.loc[3,'full_text_file'] + '/' + metadata.loc[3,'sha'].split(';')[0] + '.json'
[text['text'] for text in json.loads(open(path).read())['abstract']]

['Middle-aged female identical twins, one of whom had systemic lupus erythematosus (SLE), were evaluated for immunologic reactivity to previous antigenic challenges, including primary immunization with a foreign antigen, keyhole limpet hemocyanin (KLH). These two women had lived together for all of their 58 years and neither was receiving anti-inflammatory or immunosuppressive drugs at the time of these studies. Both twins demonstrated comparable 7s and 19s humoral antibody response to KLH, as well as similar viral antibody titers. However, the twin with SLE was anergic to common antigens, streptokinase-streptodornase, Trichophyton and Candida; furthermore delayed hypersensitivity to KLH did not develop after immunization. This observed discrepancy between humoral and cellular immunity in genetically similar subjects may be significant in the pathogenesis of SLE.',
 'Reports of an increased incidence of systemic lupus erythematosus (SLE), other connective tissue diseases, and serologic

In [8]:
metadata_with_file = metadata[metadata.sha.notnull()]
metadata_with_file.shape

(28462, 15)

In [9]:
metadata_without_file = metadata[~metadata.sha.notnull()]
metadata_without_file.shape

(15758, 15)

In [10]:
#For files which have json object - 
#    if full_text exists:
#       take full text
#    else
#       take abstract
texts_with_file = metadata_with_file.apply(lambda x: [text['text'] for text in json.loads(open(('./' + x['full_text_file'] + '/' + x['full_text_file'] + '/' + x['sha'].split(';')[0] + '.json')).read())['abstract']] if x['has_full_text']==False else [text['text'] for text in json.loads(open('./' + x['full_text_file'] + '/' + x['full_text_file'] + '/' + x['sha'].split(';')[0] + '.json').read())['body_text']] ,axis=1)

In [11]:
# Get paragraphs out of file-wise texts
paras = [para for paras in texts_with_file for para in paras]

In [12]:
# Getting memory error unfortunately when trying to convert paragraph into sentences
from textblob import TextBlob
blob = TextBlob(" ".join(paras))
sentences = [item.raw for item in blob.sentences]

In [13]:
len(sentences)

5965461

In [4]:
#saving sentences
#Load sentences
with open('./intermediate_files/sentences_from_files_with_json.pkl', 'wb') as f:
    pickle.dump(sentences, f)


In [3]:
with open('./intermediate_files/sentences_from_files_with_json.pkl', 'rb') as f:
    sentences = pickle.load(f)
print("loaded {} sentences".format(len(sentences)))

loaded 5965461 sentences


### Fasttext

In [12]:
import sister
embedder = sister.MeanEmbedding(lang="en")

Loading model...


In [26]:
from tqdm import tqdm_notebook as tqdm

In [ ]:
dict_embeddings = {}
for i in tqdm(range(len(sentences))):
    dict_embeddings[sentences[i]] = embedder(sentences[i])

array([ 0.12065388, -0.07750233,  0.18022586, -0.19695258,  0.09707139,
        0.16337198, -0.116189  ,  0.0102306 ,  0.13303241,  0.07867083,
        0.01429848,  0.03898379, -0.14972755,  0.139046  , -0.15605552,
        0.02578273, -0.1125486 ,  0.15283372, -0.00383568, -0.06921934,
        0.17681456,  0.33064702, -0.1186348 , -0.11882814, -0.00291477,
       -0.04755168,  0.11235552, -0.13397461,  0.00554298,  0.04303724,
        0.06380208,  0.11188086,  0.00511935,  0.20433584, -0.13329247,
       -0.14840195, -0.07129596,  0.00105291, -0.00963586,  0.02508393,
        0.05040146, -0.19475326,  0.13310662, -0.11146209,  0.11309413,
        0.07005211, -0.0837797 ,  0.05551897,  0.05165002,  0.1138142 ,
       -0.00826896,  0.02000929, -0.12975773,  0.04007236, -0.12547356,
        0.03074553,  0.16206928,  0.06770144,  0.07248233, -0.01023317,
       -0.23393972,  0.14831501, -0.12260765, -0.06599101, -0.13198979,
       -0.07840385, -0.19913028, -0.04502749,  0.03930811,  0.01

In [37]:
len(s)

5965461

In [39]:
dict_embeddings = dict(zip(sentences,sentence_embeddings))

In [41]:
import gc
gc.collect()

28

In [52]:
from sklearn.decomposition import PCA

In [53]:
pca = PCA(n_components=50)

In [54]:
sentence_embeddings_50_dim = pca.fit(sentence_embeddings)

MemoryError: Unable to allocate 13.3 GiB for an array with shape (5965461, 300) and data type float64

In [43]:
#save embeddings
with open('./intermediate_files/sentence_embeddings_fasttext.pkl', 'wb') as f:
    pickle.dump(sentence_embeddings, f)

MemoryError: 

In [44]:
q1 = "seasonality of transmission"

In [45]:
q1_embedded = embedder(q1)

In [46]:
q1_embedded

array([ 0.16235054, -0.30135378, -0.09185278, -0.19099635,  0.0775651 ,
        0.22391896, -0.18385087,  0.08009677,  0.15056495, -0.01638009,
        0.17891079,  0.05597444, -0.0659718 ,  0.05182942, -0.24176426,
        0.00510313, -0.17250423,  0.08795262, -0.08259635,  0.02472942,
        0.34198776,  0.29714352, -0.10302507, -0.05614026, -0.05325069,
       -0.18440421,  0.13519837, -0.14561729, -0.25528714, -0.01748819,
       -0.0060047 ,  0.07615267, -0.21097238,  0.09350184,  0.1582446 ,
       -0.06087643, -0.01116056,  0.19759   , -0.14976196,  0.01993369,
       -0.07470035, -0.1127196 ,  0.00394853, -0.33341274,  0.3634481 ,
        0.07540161, -0.16946094,  0.09318512,  0.16739559,  0.11675748,
       -0.08487552, -0.04267214, -0.33920565, -0.07577255,  0.00591114,
       -0.0398283 ,  0.11590744,  0.16220142,  0.07651661,  0.2956772 ,
       -0.31280443,  0.3273605 , -0.07441439, -0.05649571, -0.24788652,
        0.03406953, -0.6457343 ,  0.25816882, -0.10517436,  0.09

In [48]:
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity

In [51]:
np.argmax(cosine_similarity(q1_embedded,sentence_embeddings))

MemoryError: Unable to allocate 6.67 GiB for an array with shape (5965461, 300) and data type float32

In [6]:
model_file = './infersent/infersent1.pkl'`

In [7]:
from models import InferSent

In [11]:
MODEL_PATH = './infersent/infersent1.pkl'
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [11]:
W2V_PATH = 'glove/glove.840B.300d.txt'
infersent.set_w2v_path(W2V_PATH)

In [12]:
%%time
#build vocab

#infersent.build_vocab(sentences, tokenize=True)
infersent.build_vocab(sentences, tokenize=True)

Found 379102(/1658613) words with w2v vectors
Vocab size : 379102
Wall time: 46min 42s


In [14]:
#saving infersent
with open('./infersent/infersent_with_vocab.pkl', 'wb') as f:
    pickle.dump(infersent, f)

In [4]:
#Loading infersent
with open('./infersent/infersent_with_vocab.pkl', 'rb') as f:
    infersent = pickle.load(f)

In [5]:
%%time
dict_embeddings = {}
for i in tqdm(range(len(sentences))):
    dict_embeddings[sentences[i]] = infersent.encode([sentences[i]], tokenize=True)

  1%|▍                                                                   | 43854/5965461 [2:12:43<463:25:14,  3.55it/s]

KeyboardInterrupt: 

In [219]:
# embed the paragraphs(ideally should be sentences)
embeddings = infersent.encode(paras, tokenize=True)

In [18]:
#Pre process fucntion
import re
import nltk
import string
from nltk.corpus import stopwords
stop = stopwords.words('english')
nltk.download('wordnet')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

import enchant
d = enchant.Dict("en_US")

nltk.download('words')
words = set(nltk.corpus.words.words())

def preprocess(data):
    print(1)
    #lower case
    text_lower = data['Text'].apply(lambda x : str(x).lower())
    data['Text'] = text_lower
    #remove non english terms
    print(2)
    data.loc[:,'Text'] = data['Text'].apply(lambda x: " ".join(w for w in str(x).split() if d.check(w.lower())))
    
    #remove numbers and special characters
    print(3)
    data.loc[:,'Text'] = data['Text'].apply(lambda x: x.replace(r"[^a-zA-Z ]+", " ").strip())
    
    data.loc[:,'Text'] = data['Text'].apply(lambda x:x.translate(str.maketrans('', '', string.punctuation)))
    
    data.loc[:,'Text'] = data['Text'].apply(lambda x: ''.join(i for i in x if not i.isdigit()))
    
    
    
    #remove punctuation
    #data.loc[:,'Text'] = data['Text'].str.replace('[^\w\s]','')
    
    #remove sentences with less than or equal to three words
    print(4)
    def remove_less_than_3_word_sentences(data):
        count = data['Text'].str.split().str.len()
        data = data[(count>3)]
        return data
    data = remove_less_than_3_word_sentences(data)
    
    #remove stop words
    print(5)
    data.loc[:,'Text'] = data['Text'].apply(lambda x: " ".join(item for item in w_tokenizer.tokenize(x) if item not in stop))

    
    #lemmatize
    print(6)
    data.loc[:,'Text'] = data['Text'].apply(lambda x: " ".join(lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(str(x))))
    
    return data

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\puboggavarapu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\puboggavarapu\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


## CAUTION

In [55]:
# #save changes to git
# import os
# from subprocess import check_call
# from shlex import split

# def post_save( os_path):
#     """post-save hook for doing a git commit / push"""
#     check_call(split('git add {}'.format(filename)), cwd=workdir)
#     check_call(split('git commit -m "notebook save" {}'.format(filename)), cwd=workdir)
#     check_call(split('git push'), cwd=workdir)
    
    
# file_full_path = r'C:\Users\puboggavarapu\Desktop\skills&studies\kaggle\github\covid19\Baseline_Model.ipynb'
# post_save(file_full_path)

NameError: name 'filename' is not defined